In [1]:
# STEP 4. Extracted Session Cleaning
# Methods to dectect if the extraction is correct, apply to all files including error tags files
# First try out with error flag free files
import numpy as np
import nltk
import string
import requests
import re
import glob
import os, sys
import datetime
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import pandas as pd
import xlrd
import csv
import time
import dask
import dask.dataframe as dd
from bs4 import BeautifulSoup
from sec_edgar_downloader import Downloader
from sklearn.feature_extraction.text import TfidfVectorizer
import shutil
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
import nltk
nltk.download('stopwords')
dl = Downloader("/home/hongzhuoqiao/10K_Projects/sec_filings")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hongzhuoqiao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def clean_ticker(tic):
    
    if re.match('\w+[.]\w+', str(tic)):
        if re.match('\w+[.]\d+', str(tic)):
            tic = ''.join([i for i in tic if not i.isdigit()])
            tic = ''.join([i for i in tic if not i is "."])    
        else:
            tic = ''.join([i for i in tic if not i is "."])

    if re.match('\w+[.]', str(tic)):        
        tic = ''.join([i for i in tic if not i is "."]) 
    return tic

company_df = pd.read_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/multiple_observations.txt", sep='\t')
tc_df = company_df[['cusip','tic']].drop_duplicates()
tc_df['ticker'] = tc_df['tic'].apply(clean_ticker)
tc_df.to_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/tc.csv")
tc_df.head()

,cusip,tic,ticker
0,000361105,AIR,AIR
13,000886309,ADCT.1,ADCT
17,001058205,IWKS,IWKS
19,020813101,ALO.2,ALO
20,02376R102,AAL,AAL


In [11]:
# Get companies in the original list
result_df = pd.read_csv("result.csv", index_col=0)
print("{} company names provided in the original list.\n".format(len(result_df)))

# Get companies have 10k files extracted
havefile_df = result_df[result_df['totals']!=0]
print("{} company have 10k files extracted.\n".format(len(havefile_df)))

# Get companies have 10k file and no error flag (not necessary error free
flag_free_df = havefile_df[havefile_df['errors']==0]
print("{} companies have no error flag. (Not neccessary error free.)\n".format(len(flag_free_df)))
flag_free_df.head()

19825 company names provided in the original list.

7842 company have 10k files extracted.

5697 companies have no error flag. (Not neccessary error free.)



,cusip,tic,ticker,extraction_info,totals,extracted,errors
0,000361105,AIR,AIR,"26, 26, 0, []",26,26,0
13,000886309,ADCT.1,ADCT,"18, 18, 0, []",18,18,0
19,020813101,ALO.2,ALO,"14, 14, 0, []",14,14,0
32,125141101,CECE,CECE,"19, 19, 0, []",19,19,0
81,723484101,PNW,PNW,"27, 27, 0, []",27,27,0


In [13]:
# Read item1 section txt file
ticker = 'AAPL'
item1_files_path = "/home/hongzhuoqiao/10K_Projects/sec_filings/item1_section/"+ticker+"/*.txt"
item1_files = glob.glob(item1_files_path)
item1_files = item1_files[:1]

for file in item1_files:
    filename = os.path.basename(file)
    f = open(file, 'r')
    print(filename)
    print(f.read()[:500])
    # TODO: first check the word count
    


AAPL-15.txt
>Item 1.
Business  Company Background 
The Company designs, manufactures and markets mobile communication and media devices, personal computers and portable digital music players, and sells a
variety of related software, services, accessories, networking solutions and third-party digital content and applications. The Companys products and services include iPhone®, iPad®, Mac®, iPod®, Apple
Watch®, Apple TV®, a portfolio of consumer and professional software applications, iOS, OS X® and watchOS


In [10]:
# Read Initial Sim_matrix Files from folders
sim_matrix_files_path = "/home/hongzhuoqiao/10K_Projects/sec_filings/tfidf_similarity/sim_matrix/*.csv"
sim_matrix_files = glob.glob(sim_matrix_files_path)
sim_matrix_files = sim_matrix_files[:1]

for file in sim_matrix_files:
    filename = os.path.basename(file)
    tickername = filename[:-11]
    sim_df = pd.read_csv(file, index_col=0)

sim_df

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
2017,1.000000,0.972623,0.720812,0.726446,0.737313,0.734922,0.710277,0.710384,0.688697,0.641284
2016,0.972623,1.000000,0.742760,0.749153,0.750230,0.737251,0.716011,0.716289,0.696303,0.652791
2015,0.720812,0.742760,1.000000,0.972277,0.926175,0.874024,0.844240,0.842617,0.834738,0.799455
2014,0.726446,0.749153,0.972277,1.000000,0.938843,0.881524,0.849140,0.847906,0.839444,0.809318
2013,0.737313,0.750230,0.926175,0.938843,1.000000,0.940379,0.903517,0.899607,0.881640,0.844096
2012,0.734922,0.737251,0.874024,0.881524,0.940379,1.000000,0.962616,0.958185,0.934107,0.881648
2011,0.710277,0.716011,0.844240,0.849140,0.903517,0.962616,1.000000,0.994890,0.976195,0.929543
2010,0.710384,0.716289,0.842617,0.847906,0.899607,0.958185,0.994890,1.000000,0.982936,0.936971
2009,0.688697,0.696303,0.834738,0.839444,0.881640,0.934107,0.976195,0.982936,1.000000,0.958448
2008,0.641284,0.652791,0.799455,0.809318,0.844096,0.881648,0.929543,0.936971,0.958448,1.000000


In [6]:
# Similarity Function with Preprocessing
# doc_test_list =['  <Hello% my_World 2020! BYE    2019@!!   ', 'item.1A HELLO the WORLD 2019.']
def preprocess(docs):
    sepcial_stopwords =['item','items','itemsa'] # create sepcial stopwords set
    docs = [doc.lower() for doc in docs] # lowercase
    table = str.maketrans('','', string.punctuation) # remove punctuation 1
    docs = [doc.translate(table) for doc in docs] # remove punctuation 2
    docs = [re.sub(r'\d+', 'num', doc) for doc in docs] # replace all # to 'num'
    docs = [re.sub('[^\w\s]', '',doc) for doc in docs] # remove special char
    docs = [re.sub('_', '', doc) for doc in docs] # remove underscore
    docs = [re.sub('\s+',' ', doc) for doc in docs] # change whitespaces to one space
    stopwords = set(nltk.corpus.stopwords.words('english')+sepcial_stopwords) # create stopwords set with special stopword if have any 1
    docs = [[word for word in doc.split() if word not in stopwords] for doc in docs]  # create stopwords set with special stopword if have any 2
    stemmer = nltk.stem.PorterStemmer() # stemming 1
    docs = [" ".join([stemmer.stem(word) for word in doc]) for doc in docs] # stemming 2
    docs = [doc.strip() for doc in docs]
    return docs

def similarity_calculation(ticker):
    print("\n>>> Start meausuring company {}".format(ticker))
    path_header = "/home/hongzhuoqiao/10K_Projects/sec_filings/item1_section"
    path_for_company_folder = path_header + "/" + ticker +"/*.txt"
    files = glob.glob(path_for_company_folder)
    file_list =[]
    filename_list =[]
    for file in files:
        f = open(file, 'r')
        filename = os.path.basename(file)
        file_list.append(f.read())
        filename_list.append(filename[-6:-4])

    filename_list = [datetime.datetime.strptime(f,'%y').strftime('%Y') for f in filename_list]
    file_list = preprocess(file_list)
    tfidf = TfidfVectorizer(stop_words='english', strip_accents='ascii').fit_transform(file_list)
    pairwise_similarity = tfidf*tfidf.T
    sim_df = pd.DataFrame(pairwise_similarity.toarray(), columns = filename_list, index = filename_list)
    sim_df.sort_index(axis=0, ascending=False, inplace=True)
    sim_df.sort_index(axis=1, ascending=False, inplace=True)
    #sim_df.to_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/tfidf_similarity/sim_matrix/"+ticker+"_matrix.csv")
    print (">>>) Got similarity matrix for {}".format(ticker))
    
    
    # last_year_sim_list =[]
    # for i in range(0,len(sim_df)-1):
    #     last_year_sim_list.append(sim_df.iloc[i,i+1])
    # last_year_sim_df = pd.DataFrame(last_year_sim_list, index= sim_df.columns[:-1], columns=[ticker])
    # last_year_sim_df.to_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/tfidf_similarity/sim_pair/"+ticker+"_pair.csv")
    # Got similarity pairs for {}".format(ticker))
    # if (last_year_sim_list):
    #     return 1
    # else:
    #     return 0
    
    return sim_df

#similarity_calculation(ticker).to_csv('apple_sim_example.csv')

In [17]:
# detect and correct extraction
# TODO: methods to detect abnormality in extracted item1 files (using similarity score, length)


3

In [80]:
# Last_3y_sim calculation from existing sim_matrix
def last_3y_similarity(sim_df,tickername):
    last_3y_sim_list =[]
    for i in range(0,len(sim_df)-3):
        last_3y_sim_list.append((np.sum(sim_df.iloc[i,i+1] + sim_df.iloc[i,i+2] + sim_df.iloc[i,i+3]))/3)
    last_3y_sim_df = pd.DataFrame(last_3y_sim_list, index = sim_df.columns[:-3],columns=[tickername])
    return last_3y_sim_df
    
# Generate 3y_sim files
sim_matrix_files_path = "/home/hongzhuoqiao/10K_Projects/sec_filings/tfidf_similarity/sim_matrix/*.csv"
sim_matrix_files = glob.glob(sim_matrix_files_path)
sim_3y_df_list =[]

for file in sim_matrix_files:
    filename = os.path.basename(file)
    tickername = filename[:-11]
    sim_df = pd.read_csv(file, index_col=0)
    sim_3y_df = last_3y_similarity(sim_df,tickername)
    sim_3y_df_list.append(sim_3y_df)
    print(">>> {} sim_3y appended".format(tickername))
    sim_3y_df.to_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/tfidf_similarity/sim_3y/"+tickername+"_3y.csv")
    print(">>> Got 3y similarity pair for {}".format(tickername))

sim_3y_df = pd.concat(sim_3y_df_list, axis=1)
sim_3y_df.head()

rity pair for SOI
>>> KYAK sim_3y appended
>>> Got 3y similarity pair for KYAK
>>> CPTA sim_3y appended
>>> Got 3y similarity pair for CPTA
>>> EQT sim_3y appended
>>> Got 3y similarity pair for EQT
>>> BR sim_3y appended
>>> Got 3y similarity pair for BR
>>> TCBI sim_3y appended
>>> Got 3y similarity pair for TCBI
>>> RCHA sim_3y appended
>>> Got 3y similarity pair for RCHA
>>> NBTF sim_3y appended
>>> Got 3y similarity pair for NBTF
>>> SWX sim_3y appended
>>> Got 3y similarity pair for SWX
>>> KED sim_3y appended
>>> Got 3y similarity pair for KED
>>> AMCO sim_3y appended
>>> Got 3y similarity pair for AMCO
>>> VLNCQ sim_3y appended
>>> Got 3y similarity pair for VLNCQ
>>> CVGW sim_3y appended
>>> Got 3y similarity pair for CVGW
>>> HCFT sim_3y appended
>>> Got 3y similarity pair for HCFT
>>> SUAI sim_3y appended
>>> Got 3y similarity pair for SUAI
>>> PACI sim_3y appended
>>> Got 3y similarity pair for PACI
>>> TFS sim_3y appended
>>> Got 3y similarity pair for TFS
>>> BAX sim_3y a

,MDXG,CFNL,HRLY,SWCH,IPSU,PLAG,LORL,ANSW,WPG,EDUC,...,LITE,AROC,CVD,MHGC,OCR,CSGS,ARCH,NGSX,RNOW,SFS
1997,NaN,NaN,NaN,NaN,0.956083,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,NaN,NaN,0.912484,NaN,0.925148,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.949402,NaN,NaN,NaN,NaN,NaN
1999,NaN,NaN,0.928833,NaN,0.950455,NaN,0.831361,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.957610,NaN,NaN,NaN,NaN,NaN
2000,NaN,NaN,0.750240,NaN,0.976917,NaN,0.872993,NaN,NaN,0.919292,...,NaN,NaN,0.821110,NaN,0.965917,0.920033,NaN,NaN,NaN,NaN
2001,NaN,NaN,0.865913,NaN,0.866062,NaN,0.911002,NaN,NaN,0.958608,...,NaN,NaN,0.848734,NaN,0.897582,0.947296,0.728552,NaN,NaN,NaN


In [7]:
# Last_5y_sim calculation from existing sim_matrix
def last_5y_similarity(sim_df,tickername):
    last_5y_sim_list =[]
    for i in range(0,len(sim_df)-5):
        last_5y_sim_list.append((np.sum(sim_df.iloc[i,i+1] + sim_df.iloc[i,i+2] + sim_df.iloc[i,i+3] + sim_df.iloc[i,i+4] + sim_df.iloc[i,i+5]))/5)
    last_5y_sim_df = pd.DataFrame(last_5y_sim_list, index = sim_df.columns[:-5],columns=[tickername])
    return last_5y_sim_df
    
# Generate 5y_sim files
sim_matrix_files_path = "/home/hongzhuoqiao/10K_Projects/sec_filings/tfidf_similarity/sim_matrix/*.csv"
sim_matrix_files = glob.glob(sim_matrix_files_path)
sim_5y_df_list =[]

for file in sim_matrix_files:
    filename = os.path.basename(file)
    tickername = filename[:-11]
    sim_df = pd.read_csv(file, index_col=0)
    sim_5y_df = last_5y_similarity(sim_df,tickername)
    sim_5y_df.to_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/tfidf_similarity/sim_5y/"+tickername+"_5y.csv")
    print("\n>>> Saved 5y similarity pair for {}".format(tickername))

    sim_5y_df_list.append(sim_5y_df)
    print(">>> {} sim_5y appended".format(tickername))
    
sim_5y_df = pd.concat(sim_5y_df_list, axis=1)
sim_5y_df.head()

rity pair for TFS
>>> TFS sim_5y appended

>>> Saved 5y similarity pair for BAX
>>> BAX sim_5y appended

>>> Saved 5y similarity pair for PAYS
>>> PAYS sim_5y appended

>>> Saved 5y similarity pair for VITL
>>> VITL sim_5y appended

>>> Saved 5y similarity pair for ALTH
>>> ALTH sim_5y appended

>>> Saved 5y similarity pair for ISYS
>>> ISYS sim_5y appended

>>> Saved 5y similarity pair for FLTX
>>> FLTX sim_5y appended

>>> Saved 5y similarity pair for MCBF
>>> MCBF sim_5y appended

>>> Saved 5y similarity pair for UMRX
>>> UMRX sim_5y appended

>>> Saved 5y similarity pair for ZCOR
>>> ZCOR sim_5y appended

>>> Saved 5y similarity pair for CHZS
>>> CHZS sim_5y appended

>>> Saved 5y similarity pair for XOM
>>> XOM sim_5y appended

>>> Saved 5y similarity pair for ABR
>>> ABR sim_5y appended

>>> Saved 5y similarity pair for MACK
>>> MACK sim_5y appended

>>> Saved 5y similarity pair for IRTC
>>> IRTC sim_5y appended

>>> Saved 5y similarity pair for TNF
>>> TNF sim_5y appended

>>> S

,MDXG,CFNL,HRLY,SWCH,IPSU,PLAG,LORL,ANSW,WPG,EDUC,...,LITE,AROC,CVD,MHGC,OCR,CSGS,ARCH,NGSX,RNOW,SFS
1999,NaN,NaN,NaN,NaN,0.931265,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000,NaN,NaN,0.731821,NaN,0.948314,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.944469,NaN,NaN,NaN,NaN,NaN
2001,NaN,NaN,0.850927,NaN,0.865451,NaN,0.821383,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.876367,NaN,NaN,NaN,NaN,NaN
2002,NaN,NaN,0.724937,NaN,0.880448,NaN,0.872888,NaN,NaN,0.949164,...,NaN,NaN,0.826216,NaN,0.912329,0.890936,NaN,NaN,NaN,NaN
2003,NaN,NaN,0.779510,NaN,0.900157,NaN,0.919951,NaN,NaN,0.960047,...,NaN,NaN,0.858767,NaN,0.938945,0.830598,0.826603,NaN,NaN,NaN


In [8]:
sim_5y_df.head()

,MDXG,CFNL,HRLY,SWCH,IPSU,PLAG,LORL,ANSW,WPG,EDUC,...,LITE,AROC,CVD,MHGC,OCR,CSGS,ARCH,NGSX,RNOW,SFS
1999,NaN,NaN,NaN,NaN,0.931265,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000,NaN,NaN,0.731821,NaN,0.948314,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.944469,NaN,NaN,NaN,NaN,NaN
2001,NaN,NaN,0.850927,NaN,0.865451,NaN,0.821383,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.876367,NaN,NaN,NaN,NaN,NaN
2002,NaN,NaN,0.724937,NaN,0.880448,NaN,0.872888,NaN,NaN,0.949164,...,NaN,NaN,0.826216,NaN,0.912329,0.890936,NaN,NaN,NaN,NaN
2003,NaN,NaN,0.779510,NaN,0.900157,NaN,0.919951,NaN,NaN,0.960047,...,NaN,NaN,0.858767,NaN,0.938945,0.830598,0.826603,NaN,NaN,NaN


In [9]:
#sim_3y_df.T.merge(result_df[['cusip','ticker']], right_on='ticker', left_index=True, how='left').to_csv('sim_3y_03181800.csv')
sim_5y_df.T.merge(result_df[['cusip','ticker']], right_on='ticker', left_index=True, how='left').to_csv('sim_5y_03191500.csv')

In [10]:
sim_5y_df.T.merge(result_df[['cusip','ticker']], right_on='ticker', left_index=True, how='left')

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,cusip,ticker
129378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.893848,0.871371,0.891866,0.739081,0.778423,NaN,NaN,NaN,602496101,MDXG
86052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.896989,0.998636,0.99827,0.998842,0.998308,NaN,NaN,NaN,14149F109,CFNL
7354,NaN,0.731821,0.850927,0.724937,0.77951,0.832559,0.886303,0.888779,0.923724,0.946044,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,427398102,HRLY
63709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87105L104,SWCH
22438,0.931265,0.948314,0.865451,0.880448,0.900157,0.924124,0.925015,0.920732,0.315516,0.808796,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,453096208,IPSU
106721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.985569,0.984722,0.98843,0.988616,0.977442,0.97969,0.704007,NaN,72703U102,PLAG
70462,NaN,NaN,0.821383,0.872888,0.919951,0.889137,0.891107,0.918215,0.892809,0.800848,...,0.890137,0.918743,0.94729,0.957628,0.980919,0.987508,0.984919,NaN,543881106,LORL
107262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03662X100,ANSW
37476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.948314,93964W108,WPG
5147,NaN,NaN,NaN,0.949164,0.960047,0.97988,0.885563,0.852028,0.720728,0.776679,...,0.493487,0.615057,0.740411,0.477096,0.472855,0.574592,0.697602,NaN,281479105,EDUC


In [48]:
# Clean exsiting file section
# Check if company's file need to be modified
company_item1_path = "/home/hongzhuoqiao/10K_Projects/sec_filings/item1_section/"+ticker+"/*.txt"
item1_files = glob.glob(company_item1_path)
wrong_item1_list =[]
for file in item1_files:
    f = open(file, 'r')
    # print('\n')    
    if len(f.read())<100:
        fileyear = os.path.basename(file)[-6:-4]
        print(fileyear)
        wrong_item1_list.append(fileyear)
        print(">>> Wrong Extraction, please check!")
wrong_item1_list

04
>>> Wrong Extraction, please check!
03
>>> Wrong Extraction, please check!
00
>>> Wrong Extraction, please check!


['04', '03', '00']

In [50]:
sec_files_path = "/home/hongzhuoqiao/10K_Projects/sec_filings/sec_edgar_filings/"+ticker+"/10-K/*.txt"
sec_files = glob.glob(sec_files_path)
wrongfile_sec_list = []
for file in sec_files:
    if (os.path.basename(file)[11:13] in wrong_item1_list):
        #print(os.path.basename(file))
        wrongfile_sec_list.append(os.path.basename(file))
wrongfile_sec_list

['0001193125-04-216917.txt',
 '0000912057-00-002160.txt',
 '0001193125-03-098260.txt']

In [53]:
# test single year cell
sec_file_name = wrongfile_sec_list[0]
single_file_path = "/home/hongzhuoqiao/10K_Projects/sec_filings/sec_edgar_filings/"+ticker+"/10-K/"+ sec_file_name

with open(single_file_path, "r") as f:
    raw_10k = f.read()
print(raw_10k[0:500])

-----BEGIN PRIVACY-ENHANCED MESSAGE-----
Proc-Type: 2001,MIC-CLEAR
Originator-Name: webmaster@www.sec.gov
Originator-Key-Asymmetric:
 MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen
 TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB
MIC-Info: RSA-MD5,RSA,
 HMMHbyl4GDt4PjuVYdyj2WyWb9UkQEZ5jUYAYKzdJSyR7fZZ69FEgNMkkWxJocox
 CpUp2ZQA4cePYfNdFERqYA==

<SEC-DOCUMENT>0001193125-04-216917.txt : 20041221
<SEC-HEADER>0001193125-04-216917.hdr.sgml : 20041221
<ACCEPTANCE-DATETIME>200


In [56]:
## adapted from the "single_company_item1_extration.ipynb"
doc_start_pattern = re.compile(r'<DOCUMENT>')
doc_end_pattern = re.compile(r'</DOCUMENT>')
# Regex to find <TYPE> tag prceeding any characters, terminating at new line
type_pattern = re.compile(r'<TYPE>[^\n]+')
doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]
doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]

#regex = re.compile(r'(Item(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0,1})|(>Item(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0,1})|(ITEM(\s|&#160;|&nbsp;)(1A|1B|1|2).{0,1})')
regex = re.compile(r'(Item(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0})|(>Item(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0})|(ITEM(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0})')

# Create a loop to go through each section type and save only the 10-K section in the dictionary
document = {}
doc = ''
extract_failed_flag = 0
for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
    
    if (doc_type =='10-K'):
        document[doc_type] = raw_10k[doc_start:doc_end]    
        # Use finditer to math the regex
        matches = regex.finditer(document['10-K'])
        doc = '10-K'    
    elif (doc_type =='10-K405'):
        document[doc_type] = raw_10k[doc_start:doc_end]    
        # Use finditer to math the regex
        matches = regex.finditer(document['10-K405'])
        doc = '10-K405'
    elif (doc_type =='10-KT'):
        document[doc_type] = raw_10k[doc_start:doc_end]    
        # Use finditer to math the regex
        matches = regex.finditer(document['10-KT'])
        doc = '10-KT'

# # Create the dataframe
try:
    matches = regex.finditer(document[doc])
    test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])
    test_df.columns = ['item', 'start', 'end']
    test_df['item'] = test_df.item.str.lower()

    # Get rid of unnesesary charcters from the dataframe
    test_df.replace('&#160;',' ',regex=True,inplace=True)
    test_df.replace('&nbsp;',' ',regex=True,inplace=True)
    test_df.replace(' ','',regex=True,inplace=True)
    test_df.replace('\.','',regex=True,inplace=True)
    test_df.replace('>','',regex=True,inplace=True)
    test_df.replace('<','',regex=True,inplace=True)
    test_df.replace('\n','',regex=True,inplace=True)
    # shift dataset to find the next, next next item
    test_df['next_1'] = test_df['item'].shift(-1,fill_value=0)
    test_df['next_1_start'] = test_df['start'].shift(-1,fill_value=0)
    test_df['next_1_start'] = test_df.next_1_start.astype('int32')
    # test_df['next_1_end'] = test_df['end'].shift(-1,fill_value=0)
    # test_df['next_1_end'] = test_df.next_1_end.astype('int32')
    test_df['pre_1'] = test_df['item'].shift(1,fill_value=0)
    test_df['pre_1_start'] = test_df['start'].shift(1,fill_value=0)
    test_df['pre_1_start'] = test_df.pre_1_start.astype('int32')

except:
    extract_failed_flag = 1

print (extract_failed_flag)
print (matches)
test_df

0


,item,start,end,next_1,next_1_start,pre_1,pre_1_start
0,item1,11834,11841,item2,12285,0,0
1,item2,12285,12292,item1,18380,item1,11834
2,item1,18380,18387,item1,18876,item2,12285
3,item1,18876,18883,item1,19344,item1,18380
4,item1,19344,19351,item1,19884,item1,18876
5,item1,19884,19891,item1,20376,item1,19344
6,item1,20376,20383,item1,21217,item1,19884
7,item1,21217,21224,item1,24924,item1,20376
8,item1,24924,24931,item1,37146,item1,21217
9,item1,37146,37152,item1,162796,item1,24924


In [62]:
# manual test cell, exact page number test cell
item_1_raw = document[doc][37146:176847]
item_1_content = BeautifulSoup(item_1_raw, 'lxml')
print(item_1_content.get_text("\n")[0:1000])

Item 15 of this report. Hewlett-Packard accounted for approximately 4 percent of our total net
revenue for the fiscal year ended October 31, 2004, 5 percent for fiscal year 2003 and 8 percent for fiscal year 2002. These figures include items sold to contract manufacturers who manufacture products on Hewlett-Packards behalf. 


 
 
Test and Measurement Business 
 
 
 
Our test and measurement business provides standard and customized solutions
that are used in the design, development, manufacture, installation, deployment and operation of electronics equipment and communications networks and services. These solutions include test and measurement instruments and systems, communications
services and network monitoring, management and optimization tools, software design tools and related services. Related services include start-up assistance, instrument productivity and application services and instrument calibration and repair. We
also offer customization, consulting and optimization ser

In [282]:
# test cell for single clean company similarity calculation
ticker = 'AAPL'
path_header = "/home/hongzhuoqiao/10K_Projects/sec_filings/item1_section"
path_for_company_folder = path_header + "/" + ticker +"/*.txt"
files = glob.glob(path_for_company_folder)
file_list =[]
filename_list =[]
for file in files:
    f = open(file, 'r')
    filename = os.path.basename(file)
    file_list.append(f.read())
    filename_list.append(filename[-6:-4])

filename_list = [datetime.datetime.strptime(f,'%y').strftime('%Y') for f in filename_list]
file_list = preprocess(file_list)
tfidf = TfidfVectorizer(stop_words='english', strip_accents='ascii').fit_transform(file_list)
pairwise_similarity = tfidf*tfidf.T
sim_df = pd.DataFrame(pairwise_similarity.toarray(), columns = filename_list, index = filename_list)
sim_df.sort_index(axis=0, ascending=False, inplace=True)
sim_df.sort_index(axis=1, ascending=False, inplace=True)
#sim_df
last_year_sim_list =[]
for i in range(0,len(sim_df)-1):
    last_year_sim_list.append(sim_df.iloc[i,i+1])

last_year_sim_df = pd.DataFrame(last_year_sim_list, index= sim_df.columns[:-1], columns=[ticker])
last_year_sim_df

,AAPL
2019,0.976045
2018,0.989524
2017,0.992558
2016,0.992068
2015,0.991011
2014,0.993275
2013,0.986325
2012,0.986523
2011,0.978898
2010,0.961200


In [153]:
# test for clean text file"
ticker = 'AAPL'
path_header = "/home/hongzhuoqiao/10K_Projects/sec_filings/item1_section"
path_for_company_folder = path_header + "/" + ticker +"/*.txt"
files = glob.glob(path_for_company_folder)
file_list =[]
filename_list =[]
for file in files:
    f = open(file, 'r')
    filename = os.path.basename(file)
    file_list.append(f.read())
    filename_list.append(filename[-6:-4])
tfidf = TfidfVectorizer().fit_transform(file_list)
pairwise_similarity = tfidf*tfidf.T
print(filename_list)
sim_df = pd.DataFrame(pairwise_similarity.toarray(), columns = filename_list, index = filename_list)
sim_df

['15', '16', '96', '19', '18', '06', '09', '02', '95', '13', '00', '14', '17', '05', '10', '99', '94', '01', '07', '03', '08', '98', '97', '11', '12', '04']


,15,16,96,19,18,06,09,02,95,13,...,94,01,07,03,08,98,97,11,12,04
15,1.000000,0.993921,0.923877,0.981403,0.988762,0.961996,0.976569,0.919970,0.920285,0.992829,...,0.915845,0.933738,0.971917,0.919453,0.965786,0.931768,0.923792,0.988685,0.991400,0.920594
16,0.993921,1.000000,0.920780,0.982771,0.991488,0.959131,0.975039,0.919472,0.917037,0.991815,...,0.912398,0.933046,0.971768,0.919240,0.963717,0.927665,0.920141,0.988839,0.990097,0.920047
96,0.923877,0.920780,1.000000,0.908384,0.911677,0.929379,0.925363,0.913900,0.979465,0.922262,...,0.972001,0.939350,0.926167,0.902252,0.919718,0.966681,0.982711,0.926484,0.927091,0.900100
19,0.981403,0.982771,0.908384,1.000000,0.985963,0.962748,0.966178,0.896755,0.905355,0.979048,...,0.900566,0.911640,0.961350,0.895479,0.955478,0.917339,0.908116,0.981142,0.980836,0.897313
18,0.988762,0.991488,0.911677,0.985963,1.000000,0.952655,0.970947,0.912477,0.908262,0.986058,...,0.903436,0.925684,0.967282,0.912881,0.958449,0.917835,0.910006,0.984925,0.984469,0.912614
06,0.961996,0.959131,0.929379,0.962748,0.952655,1.000000,0.961498,0.899686,0.928675,0.962607,...,0.924409,0.915899,0.961515,0.894135,0.958156,0.944033,0.932342,0.969001,0.968055,0.898613
09,0.976569,0.975039,0.925363,0.966178,0.970947,0.961498,1.000000,0.934914,0.922839,0.979095,...,0.918413,0.944142,0.987131,0.938882,0.981202,0.935034,0.927087,0.984027,0.982665,0.940982
02,0.919970,0.919472,0.913900,0.896755,0.912477,0.899686,0.934914,1.000000,0.914162,0.920713,...,0.911467,0.973499,0.940729,0.974416,0.925528,0.923145,0.918703,0.921400,0.923269,0.958094
95,0.920285,0.917037,0.979465,0.905355,0.908262,0.928675,0.922839,0.914162,1.000000,0.919954,...,0.990066,0.938214,0.923250,0.902239,0.917899,0.962294,0.970096,0.923811,0.924683,0.899457
13,0.992829,0.991815,0.922262,0.979048,0.986058,0.962607,0.979095,0.920713,0.919954,1.000000,...,0.915060,0.932994,0.974264,0.922404,0.968299,0.931037,0.922686,0.991629,0.995195,0.924124


In [4]:
## main function with everything
## don't delete
def download_to_item1(ticker):
    file_number = dl.get("10-K", ticker)
    print ('>>> Got {} for {}'.format(file_number, ticker))
    error = 0
    count = 1
    extract_failed_flag = 0
    error_file_list = []

    if file_number:

        path_header = "/home/hongzhuoqiao/10K_Projects/sec_filings/sec_edgar_filings"
        path_for_extraction_item1 = path_header + "/" + ticker + "/10-K" +"/*.txt"
        path_for_item1_folder = "/home/hongzhuoqiao/10K_Projects/sec_filings/item1_section/" + ticker
        if not os.path.exists(path_for_item1_folder):
            os.makedirs(path_for_item1_folder)
            print (">>> Created new folder for {}".format(ticker))
        else:
            print (">>> {} folder is already exists.".format(ticker))

        print (">>> Start extracting item1 sections for {}".format(ticker))
    
        files = glob.glob(path_for_extraction_item1)

        # extract annual file for one company, loop every year in one company
       
        for file in files:
            f = open(file, 'r')
            print(" \n\n>>>Starting open files {} for {}".format(count, ticker))
            filename = os.path.basename(file)
            print (">>> the file to be extracted is {}\n".format(filename))
            newname_suffix = filename[10:13]
            file_name = ticker+newname_suffix
            
            raw_10k = f.read()    
            ######
            # Regex to find <DOCUMENT> tags
        
            doc_start_pattern = re.compile(r'<DOCUMENT>')
            doc_end_pattern = re.compile(r'</DOCUMENT>')
            # Regex to find <TYPE> tag prceeding any characters, terminating at new line
            type_pattern = re.compile(r'<TYPE>[^\n]+')

            ### as section names
            doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
            doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]
            doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]

            #regex = re.compile(r'(Item(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0,1})|(>Item(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0,1})|(ITEM(\s|&#160;|&nbsp;)(1A|1B|1|2).{0,1})')

            regex = re.compile(r'(Item(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0})|(>Item(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0})|(ITEM(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0})')

            # Create a loop to go through each section type and save only the 10-K section in the dictionary
            document = {}
            doc = ''
            for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
                
                if (doc_type =='10-K'):
                    document[doc_type] = raw_10k[doc_start:doc_end]    
                    # Use finditer to math the regex
                    matches = regex.finditer(document['10-K'])
                    doc = '10-K'                    
                    # Write a for loop to print the matches                   
                elif (doc_type =='10-K405'):
                    document[doc_type] = raw_10k[doc_start:doc_end]    
                    # Use finditer to math the regex
                    matches = regex.finditer(document['10-K405'])
                    doc = '10-K405'

                elif (doc_type =='10-KT'):
                    document[doc_type] = raw_10k[doc_start:doc_end]    
                    # Use finditer to math the regex
                    matches = regex.finditer(document['10-KT'])
                    doc = '10-KT'

            
            # Create the dataframe
            try:
                matches = regex.finditer(document[doc])
                test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])
                test_df.columns = ['item', 'start', 'end']
                test_df['item'] = test_df.item.str.lower()

                # Get rid of unnesesary charcters from the dataframe
                test_df.replace('&#160;',' ',regex=True,inplace=True)
                test_df.replace('&nbsp;',' ',regex=True,inplace=True)
                test_df.replace(' ','',regex=True,inplace=True)
                test_df.replace('\.','',regex=True,inplace=True)
                test_df.replace('>','',regex=True,inplace=True)
                test_df.replace('<','',regex=True,inplace=True)
                test_df.replace('\n','',regex=True,inplace=True)
                # shift dataset to find the next, next next item
                test_df['next_1'] = test_df['item'].shift(-1,fill_value=0)
                test_df['next_1_start'] = test_df['start'].shift(-1,fill_value=0)
                test_df['next_1_start'] = test_df.next_1_start.astype('int32')
                # test_df['next_1_end'] = test_df['end'].shift(-1,fill_value=0)
                # test_df['next_1_end'] = test_df.next_1_end.astype('int32')
                test_df['pre_1'] = test_df['item'].shift(1,fill_value=0)
                test_df['pre_1_start'] = test_df['start'].shift(1,fill_value=0)
                test_df['pre_1_start'] = test_df.pre_1_start.astype('int32')    
            
            except:
                error = error + 1
                item_1_text = ''
                error_file_list.append(filename[11:13])
                extract_failed_flag = 1

            if not extract_failed_flag:
         
                try:
                    # doc contains 1a 1b section (newr files)
                    if len(test_df[test_df['item'].str.contains('item1a')]):
                        df = test_df[((test_df['item'] == 'item1') & (test_df['next_1'] == 'item1a'))|((test_df['item'] == 'item2') & (test_df['pre_1'] == 'item1b')) | ((test_df['item'] == 'item1') & (test_df['next_1'] == 'item2')) | ((test_df['item'] == 'item2') & (test_df['pre_1'] == 'item1a'))] 
                        df = df[['item', 'start', 'end']]
                        df['next_item'] = df['item'].shift(-1,fill_value=0)
                        df['next_item_start'] = df['start'].shift(-1,fill_value=0)
                        df['next_item_start'] = df.next_item_start.astype('int32')
                        df = df[(df['item']=='item1') & (df['next_item']=='item2')]
                        df['difference'] = df['next_item_start'] - df['start']
                        df.sort_values('difference', ascending = False, inplace = True)
                        item_1_raw = document[doc][df.iloc[0]['start']:df.iloc[0]['next_item_start']]

                    # doc no 1a 1b
                    else:
                        df_1 = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='first')
                        df_1.set_index('item', inplace=True)
                        raw_1 = document[doc][df_1['start'].loc['item1']:df_1['start'].loc['item2']]
                        
                        df_2 = test_df[((test_df['item'] == 'item1') & (test_df['next_1'] == 'item2'))|((test_df['item'] == 'item2') & (test_df['pre_1'] == 'item1'))]
                        df_2 = df_2[['item', 'start', 'end']]
                        df_2['next_item'] = df_2['item'].shift(-1,fill_value=0)
                        df_2['next_item_start'] = df_2['start'].shift(-1,fill_value=0)
                        df_2['next_item_start'] = df_2.next_item_start.astype('int32')
                        df_2 = df_2[(df_2['item']=='item1') & (df_2['next_item']=='item2')]
                        df_2['difference'] = df_2['next_item_start'] - df_2['start']
                        df_2 = df_2[(df_2['item']=='item1')& (df_2['start'] < 100000)]
                        df_2.sort_values('difference', ascending = False, inplace = True)
                        raw_2 = document[doc][df_2.iloc[0]['start']:df_2.iloc[0]['next_item_start']]

                        if (len(raw_1)>=len(raw_2)):
                            item_1_raw = raw_1
                        else:
                            item_1_raw = raw_2

                    item_1_content = BeautifulSoup(item_1_raw, 'lxml')
                    item_1_text = item_1_content.get_text()
                    print(item_1_text[0:1000])

                except:
                    print ("OPOOS!...Something Wrong!")
                    error = error + 1
                    item_1_text = ''
                    error_file_list.append(filename[11:13])


                # path_for_item1_folder = "/home/hongzhuoqiao/10K_Projects/sec_filings/item1_section/" + ticker
            new_file =open(path_for_item1_folder + "/" + file_name+".txt","a")
            new_file.write(item_1_text)
        
            print(">>> New item1 new file for year {} is saved\n".format(file_name))
            count = count + 1

        try:
            shutil.rmtree("/home/hongzhuoqiao/10K_Projects/sec_filings/sec_edgar_filings/"+ticker)
            print ("{}'s folder has been deleted!".format(ticker))
        except OSError as e:
            print("No folder to be deleted or errors in deleting folder.")

    else:
        print ('>>> No 10k files for this company!')

    return file_number, count-1, error,  error_file_list


In [5]:
# df_1: 0:10
# df_2: 10:20
# df_3: 20:50
# df_4: 50:100 
# df_5: 100:200 
# df_6: 200:500 
# df_7: 500:1000 
# df_8: 1000:2000 
# df_9: 2000:2041 
# df_10: 2043:2200 
# df_11: 2200:3000  
# df_12: 3000:3500 
# df_13: 3500:4000 
# df_14: 4000:5000 
# df_15: 4000:5000 (repeated)
# df_16: 5000:6000 
# df_17: 6000:7000 
# df_18: 7000:7100 
# df_19: 7100:7500 
# df_20: 7500:8000 
# df_21: 8000:8500 
# df_22: 8500:9000 
# df_23: 9000:9500 
# df_24: 9500:10000 
# df_25: 10000:10500 
# df_26: 10500:11000 
# df_27: 11000:11500 
# df_28: 11500:12000 
# df_29: 12000:12500 
# df_30: 12500:13000 
# df_31: 13000:13500 
# df_32: 13500:13825 skip 'VVI'  
# df_33: 13826:14000 
# df_34: 14000:14500 
# df_35: 14500:15000
# df_36: 15000:15500 
# df_37: 15500:16000 
# df_38: 16000:16500 
# df_39: 16500:17000 
# df_40: 17000:17500 
# df_41: 17500:18000 
# df_42: 18000:18500 
# df_43: 18500:19000 
# df_44: 19000: # current batch 
print (len(tc_df))
print("44")
df_44 = tc_df[19000:]
df_44.head()

19827
44


,cusip,tic,ticker
149645,46429B655,FLOT,FLOT
149654,025300104,CRYO,CRYO
149664,29355A107,ENPH,ENPH
149674,56502B100,MSO.UN,MSOUN
149676,74642Q101,FLOT.,FLOT


In [16]:
# error checking test cell
# print(tc_df[tc_df['ticker']=='VVI'])
# tc_df[13825:14000]

In [6]:
df_44['extraction_info'] = df_44['ticker'].apply(lambda x: download_to_item1(x))
df_44.to_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/result_44.csv")
df_44.tail(10)

ices and systems, and we use our deep expertise to serve as trusted partners to end-users who seek customized solutions to their security needs. Allegion Principal ProductsDoor closers and controlsDoors and door systemsElectronic security productsElectronic, biometric and mobile access control systemsExit devicesLocks, locksets, portable locks, key systems and servicesTime, attendance and workforce productivity systemsOther accessoriesAc
>>> New item1 new file for year ALLE-20 is saved

 

>>>Starting open files 6 for ALLE
>>> the file to be extracted is 0001579241-15-000005.txt

>Item 1.    BUSINESSOverviewAllegion plc ("Allegion," "we," "us" or "the Company") is a leading global provider of security products and solutions that keep people safe, secure and productive. We make the world safer as a company of experts, securing the places where people thrive and we create peace of mind by pioneering safety and security. We offer an extensive and versatile portfolio of mechanical and elec

,cusip,tic,ticker,extraction_info
155965,04522R101,ASLN,ASLN,"(0, 0, 0, [])"
155968,92837L109,VIST,VIST,"(0, 0, 0, [])"
155970,92845J104,VVCIF,VVCIF,"(0, 0, 0, [])"
155974,G6700G107,NVT,NVT,"(2, 2, 0, [])"
155981,Y28895103,GRIN,GRIN,"(0, 0, 0, [])"
155985,Y62151108,NMCI,NMCI,"(0, 0, 0, [])"
155987,04274P105,CSTPF,CSTPF,"(0, 0, 0, [])"
155988,039653100,ACA,ACA,"(2, 2, 0, [])"
155994,Y1146L109,CTRM,CTRM,"(0, 0, 0, [])"
155996,46641Q399,BBUS,BBUS,"(0, 0, 0, [])"


In [19]:
tfidf = TfidfVectorizer().fit_transform(file_list)
pairwise_similarity = tfidf*tfidf.T

In [24]:
sim_df = pd.DataFrame(pairwise_similarity.toarray())
sim_df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1.000000,0.925136,0.984173,1.000000,0.974636,0.905025,0.925136,0.925136,0.973557,0.984361,...,0.968544,0.948330,0.960336,0.908735,0.968544,0.984361,1.000000,0.905025,1.000000,0.905025
1,0.925136,1.000000,0.920893,0.925136,0.932696,0.960843,1.000000,1.000000,0.920919,0.928235,...,0.918406,0.977083,0.953809,0.975538,0.918406,0.928235,0.925136,0.960843,0.925136,0.960843
2,0.984173,0.920893,1.000000,0.984173,0.964579,0.900964,0.920893,0.920893,0.981967,0.972175,...,0.975787,0.943888,0.956465,0.904770,0.975787,0.972175,0.984173,0.900964,0.984173,0.900964
3,1.000000,0.925136,0.984173,1.000000,0.974636,0.905025,0.925136,0.925136,0.973557,0.984361,...,0.968544,0.948330,0.960336,0.908735,0.968544,0.984361,1.000000,0.905025,1.000000,0.905025
4,0.974636,0.932696,0.964579,0.974636,1.000000,0.912643,0.932696,0.932696,0.962654,0.983581,...,0.958992,0.952680,0.971378,0.918273,0.958992,0.983581,0.974636,0.912643,0.974636,0.912643
5,0.905025,0.960843,0.900964,0.905025,0.912643,1.000000,0.960843,0.960843,0.900094,0.909035,...,0.897825,0.951898,0.926139,0.979847,0.897825,0.909035,0.905025,1.000000,0.905025,1.000000
6,0.925136,1.000000,0.920893,0.925136,0.932696,0.960843,1.000000,1.000000,0.920919,0.928235,...,0.918406,0.977083,0.953809,0.975538,0.918406,0.928235,0.925136,0.960843,0.925136,0.960843
7,0.925136,1.000000,0.920893,0.925136,0.932696,0.960843,1.000000,1.000000,0.920919,0.928235,...,0.918406,0.977083,0.953809,0.975538,0.918406,0.928235,0.925136,0.960843,0.925136,0.960843
8,0.973557,0.920919,0.981967,0.973557,0.962654,0.900094,0.920919,0.920919,1.000000,0.967884,...,0.991365,0.942758,0.952833,0.904448,0.991365,0.967884,0.973557,0.900094,0.973557,0.900094
9,0.984361,0.928235,0.972175,0.984361,0.983581,0.909035,0.928235,0.928235,0.967884,1.000000,...,0.963438,0.949079,0.961683,0.912824,0.963438,1.000000,0.984361,0.909035,0.984361,0.909035


In [25]:
# alternative similarity Model
def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    print (">>> covnert_tag Function\n")
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None

def doc_to_synsets(doc):
    """
    Returns a list of synsets in document.

    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.

    Args:
        doc: string to be converted
    Returns:
        list of synsets
    Example:
        doc_to_synsets('Fish are nvqjp friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """
    token = nltk.word_tokenize(doc)
    # add parts of speech to token
    tag = nltk.pos_tag(token)
    # convert nltk pos into wordnet pos
    nltk2wordnet = [(i[0], convert_tag(i[1])) for i in tag]
    # if there are no synsets in token, ignore, else put in a list
    output = [wn.synsets(i, z)[0] for i, z in nltk2wordnet if len(wn.synsets(i, z))>0]
    print (">>> doc_to_synsets Function\n")
    return output

def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """
    list1 = []
    # For each synset in s1
    for a in s1:
        # finds the synset in s2 with the largest similarity value
        list1.append(max([i.path_similarity(a) for i in s2 if i.path_similarity(a) is not None]))

    output = sum(list1)/len(list1)
    
    print (">>> similarity_score Function\n")
    return output

def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""
            # first function u need to create
    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)
            # 2nd function u need to create
    print (">>> document_path_similarity Function\n")
    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

#document_path_similarity(a02, a03)

In [ ]:
from bert_embedding import BertEmbedding

In [ ]:
# Removing Punctuation
def remove_puncuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

In [ ]:
# removing Stopwords
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [ ]:
# Stemming & Lemmatizing
lemmatizer = WordNetLemmatizer()
def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize (i) for i in text]
    return len_text

stemmer = PorterStemmer()
def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

In [ ]:
dl.Downloader()
dl.supported_filings

In [ ]:
dl.get("10-K", "AAPL")

In [ ]:


f_apple = open("apple.txt","r+")
apple =f_apple.read()
f_amazon = open("amazon.txt","r+")
amazon =f_amazon.read()
f_exxon = open("exxon.txt", "r+")
exxon = f_exxon.read()
document_path_similarity('i hate run', 'i hate apple')